# Построение классификатора

Наш классификатор будет работать таким образом: по выборке строится дистанцированный граф, для него считается хроматическое число и размер максимального независимого множества, по ним и размеру выборки какой-то алгоритм классификации делает предсказание.

Пусть принятие нулевой гипотеры -- False, а первой -- True. Тогда ошибка первого рода это $FPR =\frac{FP}{FP + TN}$, 
а мощность это $Recall = \frac{TP}{TP + FN}$

Для начала сгенерируем данные для обучения классификатора

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd

from monte_carlo_experiment import monte_carlo_experiment_for_several_parameters

In [3]:
D = 1.5                     # параметр d для dist графа
n_sampels = 20             # число итераций
lambda_h0 = 1               # lambda для H0
lambda_h1 = 1/np.sqrt(10)   # lambda для H1

In [4]:
metrics_cromatic_number = []
metrics_size_maximal_independent_set = []
data_size = []
type_dist = []

for n in tqdm(range(10, 610, 1)):
    params_h0 = {
            'n': n,
            'x': D,
            'lambda': lambda_h0,
            'distribution': 'h0'
         }
    
    params_h1 = {
            'n': n,
            'x': D,
            'lambda': lambda_h1,
            'distribution': 'h1'
         }
    
    result_h0 = monte_carlo_experiment_for_several_parameters(params_h0, n_sampels, False)
    result_h1 = monte_carlo_experiment_for_several_parameters(params_h1, n_sampels, False)

    cromatic_number_h0 = result_h0['metrics_cromatic_number']
    size_maximal_independent_set_h0 = result_h0['metrics_size_maximal_independent_set']
    cromatic_number_h1 = result_h1['metrics_cromatic_number']
    size_maximal_independent_set_h1 = result_h1['metrics_size_maximal_independent_set']
   
    type_dist_h0 = [0 for i in range(n_sampels)]
    type_dist_h1 = [1 for i in range(n_sampels)]

    metrics_cromatic_number+=cromatic_number_h0
    metrics_cromatic_number+=cromatic_number_h1
    metrics_size_maximal_independent_set+=size_maximal_independent_set_h0
    metrics_size_maximal_independent_set+=size_maximal_independent_set_h1
    type_dist+=type_dist_h0
    type_dist+=type_dist_h1
    data_size+=[n for _ in range(2*n_sampels)]
    

data = {
    "cromatic_number": metrics_cromatic_number,
    "independent_set": metrics_size_maximal_independent_set,
    "data_size": data_size,
    "type_dist": type_dist
}


100%|██████████| 600/600 [30:25<00:00,  3.04s/it]


In [5]:
df = pd.DataFrame(data)

Наши данные для обучения готовы.

## Посоздаем модельки

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score, 
                            recall_score, f1_score, 
                            confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns


def fpr_score(y_true, y_pred, pos_label=1):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[1 - pos_label, pos_label]).ravel()
    denominator = fp + tn
    fpr = fp / denominator if denominator != 0 else 0.0
    return fpr


X = df.drop("type_dist", axis=1)
y = df['type_dist']

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2,      # Размер тестовой выборки (20%)
    random_state=42,    # Для воспроизводимости
    stratify=y          # Сохраняем распределение классов
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### KNN

In [7]:
knn = KNeighborsClassifier(n_neighbors=5) 
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)

In [8]:
print("Оценка качества модели KNN:")
print("----------------------------------")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"FPR: {fpr_score(y_test, y_pred):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred):.4f}")

Оценка качества модели KNN:
----------------------------------
Recall: 0.9754
FPR: 0.0317
Accuracy: 0.9719
Precision: 0.9686
F1-score: 0.9720


Как мы видим, обычный KNN уже отлично справляется! Но попробуем другие модельки

### Логрегрессия

In [9]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(
    penalty='l2',       
    C=1.0,             
    max_iter=1000,  
    random_state=42
)
logreg.fit(X_train_scaled, y_train)

y_pred = logreg.predict(X_test_scaled)


print("Оценка качества логистической регрессии:")
print("----------------------------------")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"FPR: {fpr_score(y_test, y_pred):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred):.4f}")

Оценка качества логистической регрессии:
----------------------------------
Recall: 0.9546
FPR: 0.0312
Accuracy: 0.9617
Precision: 0.9683
F1-score: 0.9614


Recall получился чуть ниже, чем у KNN

### Случайный лес

In [10]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,      # Количество деревьев
    max_depth=None,         # Глубина деревьев (без ограничений)
    min_samples_split=2,    # Минимальное число образцов для разделения узла
    min_samples_leaf=1,     # Минимальное число образцов в листе
    max_features='sqrt',    # Количество признаков для поиска лучшего разделения
    random_state=42,
    n_jobs=-1,              # Использовать все ядра процессора
    class_weight='balanced' # Учет дисбаланса классов
)


with tqdm(total=100, desc="Training Random Forest") as pbar:
    rf.fit(X_train_scaled, y_train)
    pbar.update(100)


y_pred = rf.predict(X_test_scaled)


print("Оценка качества случайного леса:")
print("----------------------------------")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"FPR: {fpr_score(y_test, y_pred):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred):.4f}")



Training Random Forest: 100%|██████████| 100/100 [00:00<00:00, 206.87it/s]

Оценка качества случайного леса:
----------------------------------
Recall: 0.9700
FPR: 0.0342
Accuracy: 0.9679
Precision: 0.9660
F1-score: 0.9680


Метрики Recall и FPR получились хуже, чем у KNN

### Итого

| Метрика     | KNN  | LogReg      |  Random Forest  |
|-------------|----- |-------------|-----------------|
|FPR          |0.0317|   0.0312    |      0.0342     |
|Recall       |0.9754|   0.9546    |      0.9700     |
|Accuracy     |0.9719|   0.9617    |      0.9679     |

Лучше всего работает KNN. FPR чуть лучше у LogReg, но у нее сильно хуже Recall.

### Построение pipeline

In [47]:
from utils import build_dist_nx, calculate_chromatic_number, calculate_size_maximal_independent_set
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler


class GraphFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, d=1.5):
        self.d = d

    def transform(self, X, y=None):
        features = []
        for data in X:
            G = build_dist_nx(data, self.d)
            chrom_num = calculate_chromatic_number(G)
            mis_size = calculate_size_maximal_independent_set(G)
            features.append([chrom_num, mis_size, len(data)])
        return np.array(features)

    def fit(self, X, y=None):
        return self

# Pipeline: Graph features -> StandardScaler -> KNN
pipeline = Pipeline([
    ('graph_features', GraphFeatureExtractor(d=1.5)),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

